In [10]:
import os
import sys

from hdlConvertor.language import Language
from hdlConvertor import HdlConvertor

TEST_DIR = os.path.join("..", "tests", "verilog")

filenames = [os.path.join(TEST_DIR, "dff_async_reset.v"), ]
include_dirs = []
c = HdlConvertor()
# note that there is also Language.VERILOG_2005, Language.SYSTEM_VERILOG_2017 and others
d = c.parse(filenames, Language.VERILOG, include_dirs, hierarchyOnly=False, debug=True)



In [5]:
from hdlConvertor.to.verilog.verilog2005 import ToVerilog2005

tv = ToVerilog2005(sys.stdout)
tv.visit_HdlContext(d)

// http://www.asic-world.com/code/hdl_models/dff_async_reset.v
//-----------------------------------------------------
// Design Name : dff_async_reset
// File Name   : dff_async_reset.v
// Function    : D flip-flop async reset
// Coder       : Deepak Kumar Tala
//-----------------------------------------------------
module dff_async_reset (
    //-----------Input Ports---------------
    input wire data,
    // Data Input
    input wire clk,
    // Clock Input
    input wire reset,
    // Reset input
    //-----------Output Ports---------------
    //------------Internal Variables--------
    output reg q
);
    //-------------Code Starts Here---------
    always @(posedge clk, negedge reset)
        if (~reset)
            q <= 1'b0;
        else
            q <= data;
endmodule


In [8]:
# each HDL AST object __repr__ returns json like string
for o in d.objs:
    print(o)

{'__class__': 'HdlModuleDef',
 'dec': {'__class__': 'HdlModuleDec',
         'declaration_only': False,
         'doc': ' http://www.asic-world.com/code/hdl_models/dff_async_reset.v\n'
                '-----------------------------------------------------\n'
                ' Design Name : dff_async_reset\n'
                ' File Name   : dff_async_reset.v\n'
                ' Function    : D flip-flop async reset\n'
                ' Coder       : Deepak Kumar Tala\n'
                '-----------------------------------------------------\n',
         'name': 'dff_async_reset',
         'objs': [],
         'params': [],
         'ports': [{'__class__': 'HdlVariableDef',
                    'direction': 'IN',
                    'doc': '-----------Input Ports---------------\n',
                    'name': 'data',
                    'position': (9, 1, 9, 4),
                    'type': {'__class__': 'HdlName', 'val': 'wire'}},
                   {'__class__': 'HdlVariableDef',
       

In [11]:
from hdlConvertor.hdlAst import HdlModuleDef

# print port names
for o in d.objs:
    if isinstance(o, HdlModuleDef):
        for x in o.dec.ports:
            print(x.name)


data
clk
reset
q


In [21]:
from hdlConvertor.to.hdl_ast_visitor import HdlAstVisitor
from hdlConvertor.hdlAst import HdlName, HdlCall

class PortNamesToUpperCase(HdlAstVisitor):
    """
    Make port names upper case HDL AST

    :note: this is just a stupid rewrite of all variable names to upper case
    """
    def visit_HdlVariableDef(self, o):
        o.name = o.name.upper()

    def visit_HdlStmProcess(self, o):
        if o.sensitivity:
            o.sensitivity = [self.visit_iHdlExpr(s) for s in o.sensitivity]
        self.visit_iHdlStatement(o.body)

    def visit_iHdlExpr(self, o):
        if isinstance(o, HdlName):
            return HdlName(o.val.upper(), obj=o.obj)
        elif isinstance(o, HdlCall):
            o.ops = [self.visit_iHdlExpr(s) for s in o.ops]
            return o
        else:
            return super(PortNamesToUpperCase, self).visit_iHdlExpr(o)

    def visit_HdlStmIf(self, o):
        o.cond = self.visit_iHdlExpr(o.cond)
        super(PortNamesToUpperCase, self).visit_HdlStmIf(o)

    def visit_HdlStmAssign(self, o):
        o.src = self.visit_iHdlExpr(o.src)
        o.dst = self.visit_iHdlExpr(o.dst)

PortNamesToUpperCase().visit_HdlContext(d)

tv = ToVerilog2005(sys.stdout)
tv.visit_HdlContext(d)

// http://www.asic-world.com/code/hdl_models/dff_async_reset.v
//-----------------------------------------------------
// Design Name : dff_async_reset
// File Name   : dff_async_reset.v
// Function    : D flip-flop async reset
// Coder       : Deepak Kumar Tala
//-----------------------------------------------------
module dff_async_reset (
    //-----------Input Ports---------------
    input wire DATA,
    // Data Input
    input wire CLK,
    // Clock Input
    input wire RESET,
    // Reset input
    //-----------Output Ports---------------
    //------------Internal Variables--------
    output reg Q
);
    //-------------Code Starts Here---------
    always @(posedge CLK, negedge RESET)
        if (~RESET)
            Q <= 1'b0;
        else
            Q <= DATA;
endmodule
